In [ ]:
# Anataolii Frolov

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install d2l==1.0.3

In [ ]:
%%capture
pip install -U kaggle  # upgrade kaggle package (to avoid a warning)

In [ ]:
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
# !cp kaggle.json ~/.kaggle/kaggle.json           # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 21-feb-25-hse-venus-mars # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download                     # download competition dataset as a zip file
!unzip -o *.zip                                   # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

Streaming output truncated to the last 5000 lines.
  inflating: train/female/fx1051d.jpg  
  inflating: train/female/fx1055d.jpg  
  inflating: train/female/fx105b2.jpg  
  inflating: train/female/fx105b9.jpg  
  inflating: train/female/fx105e7.jpg  
  inflating: train/female/fx105f1.jpg  
  inflating: train/female/fx10637.jpg  
  inflating: train/female/fx10675.jpg  
  inflating: train/female/fx1067f.jpg  
  inflating: train/female/fx10685.jpg  
  inflating: train/female/fx1068f.jpg  
  inflating: train/female/fx106c4.jpg  
  inflating: train/female/fx10742.jpg  
  inflating: train/female/fx10826.jpg  
  inflating: train/female/fx10830.jpg  
  inflating: train/female/fx1087e.jpg  
  inflating: train/female/fx1093e.jpg  
  inflating: train/female/fx109a9.jpg  
  inflating: train/female/fx109b2.jpg  
  inflating: train/female/fx109ec.jpg  
  inflating: train/female/fx1106b.jpg  
  inflating: train/female/fx110bf.jpg  
  inflating: train/female/fx110c2.jpg  
  inflating: train/female/fx1

In [ ]:
!nvidia-smi --query-gpu=gpu_name,memory.total,memory.free,memory.used --format=csv

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os
from torch.utils.data import random_split
import torch, torchvision
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import Sequential, Flatten, Linear, LazyLinear, Dropout, AdaptiveAvgPool2d, MaxPool2d, Conv2d, AvgPool2d
import torchvision
from torchvision.models import resnet50, mobilenet_v2, mobilenet_v3_small
from torchvision.datasets import ImageFolder
import torchvision.transforms as v2
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*5): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=5, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)
tDIR, sDIR = 'train/', 'test/'

CPU times: user 5.87 s, sys: 1.19 s, total: 7.05 s
Wall time: 16 s


In [ ]:
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 300 sec. Good luck!


Code

In [ ]:
!mv test/ labels/ && mkdir test && mv labels/ test/labels/

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from d2l import torch as d2l
class DataModule(d2l.HyperParameters):
    """
    A class to handle data loading and splitting.

    Stole it from the Dl boo and changed it a little
    """
    def __init__(self, tDIR, sDIR


                 ,BS = 256 ,image_size = (224,224) ,train_size = 0.8):
        # usual part here
        self.save_hyperparameters()

        transform = v2.Compose(
                              [v2.ToTensor(),
                              v2.Resize(image_size, antialias=True)]
                               ) # transforms pics from 178 to 224
        # ImageFolder initialization
        im_folder_train = ImageFolder(tDIR, transform=transform)
        im_folder_test = ImageFolder(sDIR, transform=transform)

        tDS, vDS = random_split(im_folder_train, lengths=[train_size, 1-train_size])

        self.tDL = DataLoader(tDS, batch_size=BS, shuffle=True, pin_memory=True)
        self.vDL = DataLoader(vDS, batch_size=BS, shuffle=False, pin_memory=True)
        self.sDL = DataLoader(im_folder_test, batch_size=BS, shuffle=False, pin_memory=True)
    def train_dataloader(self):
        return self.tDL
    def val_dataloader(self):
        return self.vDL
    def test_dataloader(self):
        return self.sDL


In [ ]:
class Model(d2l.Classifier): # AlexNet is too hard to compute
    def __init__(self,lr = 0.1, num_classes = 1):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
                                  nn.LazyConv2d(96, kernel_size=11, stride=4, padding=1),
                                  nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2),
                                  nn.LazyConv2d(256, kernel_size=5, padding=2), nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=3, stride=2),
                                  nn.LazyConv2d(384, kernel_size=3, padding=1), nn.ReLU(),
                                  nn.LazyConv2d(384, kernel_size=3, padding=1), nn.ReLU(),
                                  nn.LazyConv2d(256, kernel_size=3, padding=1), nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=3, stride=2), nn.Flatten(),
                                  nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(p=0.5),
                                  nn.LazyLinear(4096), nn.ReLU(),nn.Dropout(p=0.5),
                                  nn.LazyLinear(num_classes))
        self.net.apply(d2l.init_cnn)

In [ ]:
# class Model(d2l.Classifier):
#   def __init__(self, lr = 0.1, num_classes = 1):
#       super().__init__()
#       self.save_hyperparameters()
#       self.net =  nn.Sequential(
#             nn.LazyConv2d(6, kernel_size=5, padding=2), nn.Sigmoid(),
#             nn.AvgPool2d(kernel_size=2, stride=2),
#             nn.LazyConv2d(16, kernel_size=5), nn.Sigmoid(),
#             nn.AvgPool2d(kernel_size=2, stride=2),
#             nn.Flatten(),
#             nn.LazyLinear(120), nn.Sigmoid(),
#             nn.LazyLinear(84), nn.Sigmoid(),
#             nn.LazyLinear(num_classes))
#       self.net.apply(d2l.init_cnn)

In [ ]:
class Model(d2l.Classifier): # AlexNet is too hard to compute
    def __init__(self,lr = 0.1, num_classes = 1):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
          nn.LazyConv2d(32, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Linear(128, 1),
          nn.Sigmoid()
)
        self.net.apply(d2l.init_cnn)

IndentationError: unexpected indent (<ipython-input-19-ab7a5196e0b2>, line 7)

In [ ]:
from pickle import STOP
from torch import optim
class Trainer(d2l.HyperParameters):
    """The base class for training models with data."""
    def __init__(self, max_epochs
                #  progress_board,
                 ):
        self.save_hyperparameters()
        # self.board = progress_board
        #new
        self.max_epochs = max_epochs
    def prepare_data(self, data):
        self.train_dataloader = data.train_dataloader() # x and y are initiaded in train dataloader
        self.val_dataloader = data.val_dataloader() # leftovers are transferred to val ds
        #new
        self.test_dataloader = data.test_dataloader()
        self.num_train_batches = len(self.train_dataloader) # LOOK HERE AFTER CHEKING WHAT"S UP
        self.num_val_batches = (len(self.val_dataloader) if self.val_dataloader is not None else 0) # exception for just training without val
    def prepare_model(self, model):
        model.trainer = self # this was confusing at first but then made sense, also from the book
        self.model = model
    def fit(self, model, data):
        self.prepare_data(data)
        self.prepare_model(model)
        self.optim = optim.SGD(model.parameters(), lr = 0.1) # lr was changed here
        self.criterion = nn.BCEWithLogitsLoss()
        self.epoch = 0
        # for self.epoch in range(self.max_epochs):
        #     self.fit_epoch()

        print(len(data.train_dataloader().dataset.dataset.classes))  # Number of classes

        #new
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optim, gamma=0.01)

        self.fit_epoch() # made it easier for mem not iterating it here

    def fit_epoch(self,validate = True):
      losses = []
      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

      epoch = self.max_epochs
      for e in range(epoch):
        print(f"Epoch: {e}")
        self.model.train()
        #changed
        tDL = self.test_dataloader

        for i, (image, label) in enumerate(tDL):


          image = image.to(device)
          label = label.to(device).type(torch.float32).squeeze()    # ?? shape what
          self.optim.zero_grad()
          pred = self.model(image)
          loss = self.criterion(pred, label.unsqueeze(1))
          losses.append(loss)

          if i % 10 == 0:
            print(loss)

          loss.backward()
          self.optim.step()
          self.scheduler.step()

        if validate:
          self.model.eval()

          s = 0
          for i, (image, label) in enumerate(self.val_dataloader):
            image = image.to(device)
            label = label.to(device).type(torch.float32)
            with torch.no_grad():
              val_preds = nn.functional.sigmoid(self.model(image))
              s += ((val_preds > 0.5).type(torch.int) == label.unsqueeze(1)).sum()

          print(f"Acc: {s / len(self.val_dataloader)}")

      return losses


In [ ]:
Model().layer_summary((1,1,224,224))

Conv2d output shape:	 torch.Size([1, 96, 54, 54])
ReLU output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Conv2d output shape:	 torch.Size([1, 256, 26, 26])
ReLU output shape:	 torch.Size([1, 256, 26, 26])
Linear output shape:	 torch.Size([1, 256, 26, 4096])
ReLU output shape:	 torch.Size([1, 256, 26, 4096])
Dropout output shape:	 torch.Size([1, 256, 26, 4096])
Linear output shape:	 torch.Size([1, 256, 26, 1])


In [ ]:
%%time
torch.manual_seed(0)  # always seed your experiments
torch.cuda.manual_seed_all(0)  # seed all that you can

CPU times: user 2.19 ms, sys: 1.06 ms, total: 3.25 ms
Wall time: 4.01 ms


In [ ]:
ada

NameError: name 'ada' is not defined

In [ ]:
device

NameError: name 'device' is not defined

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
num_epochs = 1

model = Model()
data = DataModule(tDIR,sDIR)
trainer = Trainer(num_epochs)


trainer.fit(model, data)

In [ ]:
vIDs = [f[0].split('/')[-1].split('.')[0] for f in sDL.dataset.imgs] # parse image IDs from file names

In [ ]:
with torch.no_grad():
    preds = []
    for img, _ in sDL:
        preds.append(nn.functional.sigmoid(pm1(img.to(device))).cpu().numpy().flatten())

In [ ]:
pY = pd.DataFrame(np.concatenate(preds, axis=0), index=vIDs, columns=['y'])   # predicted targets
ToCSV((pY>0.5)*1, '🪐-baseline')